In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import tensorflow as tf
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Activation
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.python.ops.gen_batch_ops import Batch
from tensorflow.keras.regularizers import l1_l2

2024-11-08 11:50:59.228681: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-08 11:50:59.249702: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-08 11:50:59.249721: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-08 11:50:59.249735: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-08 11:50:59.253969: I tensorflow/core/platform/cpu_feature_g

In [2]:
df_gleason = pd.read_csv(r'/home/sanven0212/PycharmProjects/MultiOmicsFYP/CTGAN_Gleason_Score_Dataset.csv')
df_gleason.shape

(290, 47)

In [3]:
df_gleason['TUMOR_STAGE'].value_counts()

TUMOR_STAGE
1    145
0    145
Name: count, dtype: int64

In [4]:
df_gleason

,GE_SPOP,GE_FOXA1,GE_CTNNB1,GE_CLPTM1L,GE_DPYSL2,GE_NEIL1,GE_PITPNM2,GE_ATM,GE_EMG1,GE_ETV3,...,DM_NEIL1,DM_SLC27A4,DM_PITPNM2,DM_PTEN,DM_EMG1,DM_ETV3,DM_BRAF,DM_NKX3-1,DM_SALL1,TUMOR_STAGE
0,0.485713,0.602396,0.875180,0.493005,0.5356774057026716,0.579175,0.332847,0.725424,0.320880,0.639029,...,0.180638,0.278463,0.658140,0.043097,0.390242,0.911493,0.248904,0.433275,0.805977,1
1,0.384733,0.555186,0.384184,0.720935,0.26589053869247437,0.615938,0.263767,0.374939,0.777121,0.112752,...,0.101729,0.165342,0.760935,0.046036,0.151117,0.879784,0.136454,0.137280,0.964510,1
2,0.501783,0.617020,0.792797,0.390719,0.7326797107663086,0.499783,0.711649,0.664329,0.408121,0.878717,...,0.389931,0.507315,0.490987,0.041370,0.462390,0.947289,0.182826,0.587181,0.624428,0
3,0.610479,0.478407,0.786356,0.235442,0.7963768589855218,0.592039,0.715180,0.838920,0.173883,0.934750,...,0.253729,0.535583,0.334942,0.022867,0.606468,0.897280,0.192746,0.619840,0.745078,0
4,0.470017,0.377998,0.824718,0.393915,0.8224050291903859,0.585371,0.481900,0.611074,0.412000,0.624647,...,0.240000,0.554207,0.381248,0.076909,0.533006,0.935659,0.319159,0.876869,0.580277,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,0.346003,0.100142,0.781156,0.439434,0.9503121996878838,0.555374,0.784326,1.000000,0.315625,0.905615,...,0.368381,0.750055,0.318052,0.023063,0.633076,0.903943,0.442787,0.809851,0.569212,0
286,0.427905,0.624984,0.860459,0.453867,0.6887381290727799,0.457943,0.530200,0.688023,0.447066,0.878165,...,0.213601,0.189264,0.558862,0.077598,0.320214,0.905402,0.398802,0.521767,0.811772,0
287,0.461165,0.642609,0.878522,0.167811,0.8020363450050354,0.394491,0.630863,0.722636,0.288655,0.967985,...,0.409150,0.589928,0.590885,0.083300,0.572138,0.873300,0.145212,0.664743,0.580831,1
288,0.350445,0.535638,0.851672,0.412014,0.5734532769864495,0.593781,0.586341,0.667384,0.419121,0.853813,...,0.135575,0.320320,0.716790,0.016695,0.320311,0.908059,0.170628,0.316081,0.886800,0


In [5]:
df_gleason.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290 entries, 0 to 289
Data columns (total 47 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   GE_SPOP      290 non-null    float64
 1   GE_FOXA1     290 non-null    float64
 2   GE_CTNNB1    290 non-null    float64
 3   GE_CLPTM1L   290 non-null    float64
 4   GE_DPYSL2    290 non-null    object 
 5   GE_NEIL1     290 non-null    float64
 6   GE_PITPNM2   290 non-null    float64
 7   GE_ATM       290 non-null    float64
 8   GE_EMG1      290 non-null    float64
 9   GE_ETV3      290 non-null    float64
 10  GE_BRAF      290 non-null    float64
 11  GE_NKX3-1    290 non-null    float64
 12  GE_ZMYM3     290 non-null    float64
 13  GE_SALL1     290 non-null    float64
 14  CNA_SPOP     290 non-null    float64
 15  CNA_TP53     290 non-null    float64
 16  CNA_FOXA1    290 non-null    float64
 17  CNA_CTNNB1   290 non-null    float64
 18  CNA_MED12    290 non-null    float64
 19  CNA_CLPT

In [6]:
df_gleason.describe()

,GE_SPOP,GE_FOXA1,GE_CTNNB1,GE_CLPTM1L,GE_NEIL1,GE_PITPNM2,GE_ATM,GE_EMG1,GE_ETV3,GE_BRAF,...,DM_NEIL1,DM_SLC27A4,DM_PITPNM2,DM_PTEN,DM_EMG1,DM_ETV3,DM_BRAF,DM_NKX3-1,DM_SALL1,TUMOR_STAGE
count,290.000000,290.000000,290.000000,290.000000,290.000000,290.000000,290.000000,290.000000,290.000000,290.000000,...,290.000000,290.000000,290.000000,290.000000,290.000000,290.000000,290.000000,290.000000,290.000000,290.000000
mean,0.457905,0.550801,0.703574,0.408914,0.623916,0.525737,0.545882,0.455565,0.659265,0.346146,...,0.237672,0.382204,0.648239,0.066771,0.403366,0.913074,0.248458,0.469021,0.800310,0.500000
std,0.123463,0.134498,0.164755,0.144876,0.145306,0.179728,0.178628,0.174876,0.251019,0.130298,...,0.118488,0.178662,0.150642,0.068204,0.150910,0.077364,0.102794,0.176909,0.134097,0.500864
min,0.035294,0.000000,0.000000,0.039983,0.282511,0.020389,0.000000,0.000000,0.000000,0.000000,...,0.007223,0.000000,0.112857,0.000000,0.059518,0.144676,0.000000,0.000000,0.271312,0.000000
25%,0.377150,0.478728,0.645207,0.315880,0.524095,0.415427,0.452220,0.349906,0.488713,0.262442,...,0.164027,0.256540,0.548170,0.035151,0.301873,0.897582,0.182924,0.348342,0.724038,0.000000
50%,0.460604,0.564148,0.727977,0.402495,0.620661,0.529684,0.576034,0.443488,0.734546,0.349664,...,0.225435,0.370714,0.656926,0.055284,0.404644,0.926409,0.233488,0.463922,0.817102,0.500000
75%,0.528724,0.630950,0.821193,0.497151,0.720054,0.654844,0.671079,0.533047,0.850358,0.414111,...,0.283160,0.503387,0.759085,0.082159,0.514365,0.950489,0.288609,0.588216,0.904789,1.000000
max,1.000000,1.000000,0.990674,0.836033,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,0.952591,0.882551,0.812506,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
df_gleason['GE_DPYSL2']

0       0.5356774057026716
1      0.26589053869247437
2       0.7326797107663086
3       0.7963768589855218
4       0.8224050291903859
              ...         
285     0.9503121996878838
286     0.6887381290727799
287     0.8020363450050354
288     0.5734532769864495
289     0.6362481406367202
Name: GE_DPYSL2, Length: 290, dtype: object

In [8]:
import re
def clean(sample):
    return ''.join(re.findall(r'\d*\.?\d*', sample))
df_gleason['GE_DPYSL2'] = df_gleason['GE_DPYSL2'].apply(clean)

In [9]:
df_gleason.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290 entries, 0 to 289
Data columns (total 47 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   GE_SPOP      290 non-null    float64
 1   GE_FOXA1     290 non-null    float64
 2   GE_CTNNB1    290 non-null    float64
 3   GE_CLPTM1L   290 non-null    float64
 4   GE_DPYSL2    290 non-null    object 
 5   GE_NEIL1     290 non-null    float64
 6   GE_PITPNM2   290 non-null    float64
 7   GE_ATM       290 non-null    float64
 8   GE_EMG1      290 non-null    float64
 9   GE_ETV3      290 non-null    float64
 10  GE_BRAF      290 non-null    float64
 11  GE_NKX3-1    290 non-null    float64
 12  GE_ZMYM3     290 non-null    float64
 13  GE_SALL1     290 non-null    float64
 14  CNA_SPOP     290 non-null    float64
 15  CNA_TP53     290 non-null    float64
 16  CNA_FOXA1    290 non-null    float64
 17  CNA_CTNNB1   290 non-null    float64
 18  CNA_MED12    290 non-null    float64
 19  CNA_CLPT

In [10]:
df_gleason['GE_DPYSL2'] = df_gleason['GE_DPYSL2'].astype(float)

In [11]:
df_gleason.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290 entries, 0 to 289
Data columns (total 47 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   GE_SPOP      290 non-null    float64
 1   GE_FOXA1     290 non-null    float64
 2   GE_CTNNB1    290 non-null    float64
 3   GE_CLPTM1L   290 non-null    float64
 4   GE_DPYSL2    290 non-null    float64
 5   GE_NEIL1     290 non-null    float64
 6   GE_PITPNM2   290 non-null    float64
 7   GE_ATM       290 non-null    float64
 8   GE_EMG1      290 non-null    float64
 9   GE_ETV3      290 non-null    float64
 10  GE_BRAF      290 non-null    float64
 11  GE_NKX3-1    290 non-null    float64
 12  GE_ZMYM3     290 non-null    float64
 13  GE_SALL1     290 non-null    float64
 14  CNA_SPOP     290 non-null    float64
 15  CNA_TP53     290 non-null    float64
 16  CNA_FOXA1    290 non-null    float64
 17  CNA_CTNNB1   290 non-null    float64
 18  CNA_MED12    290 non-null    float64
 19  CNA_CLPT

In [12]:
#Splitting the dataset into train and test split
X = df_gleason.drop('TUMOR_STAGE', axis = 1)
y = df_gleason['TUMOR_STAGE']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size = 0.33, random_state = 42)

In [13]:
#Scaling the entire dataset
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train)
X_val_scaled = sc.transform(X_val)
X_test_scaled = sc.transform(X_test)

In [14]:
X_train_scaled

array([[-0.38988674,  1.18302399,  0.26187636, ..., -0.09948441,
         0.44182739,  0.2005123 ],
       [ 0.04186783, -0.37504843,  0.13306115, ...,  0.61076263,
        -0.08342236, -0.89269167],
       [-0.33109275,  1.45557699,  0.13094357, ..., -0.60948783,
        -1.45478087,  0.56170366],
       ...,
       [-0.66818529,  0.58400073, -0.67999359, ...,  0.17001831,
        -2.06752075,  1.34296193],
       [ 0.20101174,  0.03259869,  0.31550303, ...,  0.29204723,
         0.91510524, -0.27664708],
       [-0.06124093, -0.85669239,  0.72069193, ...,  0.03416772,
         1.19057148, -3.6187956 ]])

In [15]:
X_val_scaled

array([[ 0.29194321,  0.62231362,  0.98618852, ..., -0.30369555,
        -0.2699082 ,  0.25517341],
       [-0.74789474, -0.75187395, -1.24359458, ..., -0.17610281,
        -1.64647729, -0.28718398],
       [-0.28357579,  0.95041299,  0.07110192, ...,  0.24026998,
        -1.29885418,  0.74056693],
       ...,
       [-0.13587113,  0.32162238, -0.41955192, ..., -0.02571077,
        -0.44035786,  0.94711395],
       [-0.21924051, -1.01369465, -0.96682797, ..., -0.68986376,
        -0.4890417 ,  0.08345344],
       [ 0.16191336,  0.84880208,  0.10981274, ..., -0.42872006,
        -1.13685693,  0.4179798 ]])

In [16]:
X_test_scaled

array([[-0.83489719,  0.59164684,  1.10115199, ..., -0.71674736,
        -1.60750884, -0.77803354],
       [-1.47018418,  1.69489299, -0.34773907, ...,  0.13916797,
         0.67166237,  0.50383588],
       [ 1.65810842,  0.42224056, -0.27940157, ...,  0.12081837,
         0.15640527,  0.57742573],
       ...,
       [ 0.74721228, -0.35276957, -0.7977383 , ..., -0.10947759,
         2.34982697,  0.78722063],
       [-0.279067  , -0.72086265,  0.99525404, ..., -0.1754821 ,
         1.56284232, -1.03561117],
       [ 0.03479182, -0.11712953,  0.63262026, ..., -0.1279689 ,
         0.95640153, -1.42725848]])

In [17]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train_scaled, y_train)

LogisticRegression()

In [18]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_val, lr.predict(X_val_scaled)))

              precision    recall  f1-score   support

           0       0.50      0.40      0.44        15
           1       0.47      0.57      0.52        14

    accuracy                           0.48        29
   macro avg       0.49      0.49      0.48        29
weighted avg       0.49      0.48      0.48        29



In [19]:
print(classification_report(y_test, lr.predict(X_test_scaled)))

              precision    recall  f1-score   support

           0       0.69      0.56      0.62        32
           1       0.56      0.69      0.62        26

    accuracy                           0.62        58
   macro avg       0.63      0.63      0.62        58
weighted avg       0.63      0.62      0.62        58



In [20]:
print(classification_report(y_train, lr.predict(X_train_scaled)))

              precision    recall  f1-score   support

           0       0.71      0.79      0.75        98
           1       0.78      0.70      0.74       105

    accuracy                           0.74       203
   macro avg       0.75      0.75      0.74       203
weighted avg       0.75      0.74      0.74       203



In [21]:
from xgboost import XGBClassifier
xgb_model = XGBClassifier(colsample_bytree = 0.8, learning_rate = 0.1, max_depth = 3, min_child_weight = 1, n_estimators = 25, subsample = 1.0)

In [22]:
xgb_model.fit(X_train_scaled, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=25, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [23]:
print(classification_report(y_val, xgb_model.predict(X_val_scaled)))

              precision    recall  f1-score   support

           0       0.69      0.60      0.64        15
           1       0.62      0.71      0.67        14

    accuracy                           0.66        29
   macro avg       0.66      0.66      0.65        29
weighted avg       0.66      0.66      0.65        29



In [24]:
print(classification_report(y_test, xgb_model.predict(X_test_scaled)))

              precision    recall  f1-score   support

           0       0.69      0.62      0.66        32
           1       0.59      0.65      0.62        26

    accuracy                           0.64        58
   macro avg       0.64      0.64      0.64        58
weighted avg       0.64      0.64      0.64        58



In [25]:
print(classification_report(y_train, xgb_model.predict(X_train_scaled)))

              precision    recall  f1-score   support

           0       0.89      0.99      0.94        98
           1       0.99      0.89      0.93       105

    accuracy                           0.94       203
   macro avg       0.94      0.94      0.94       203
weighted avg       0.94      0.94      0.94       203



In [26]:
from sklearn.model_selection import GridSearchCV
mod_xgb_model = XGBClassifier()
xgb_param_grid = param_grid = {
    'n_estimators': [25, 50, 100],
    'learning_rate': [0.001, 0.01, 0.1],
    'max_depth': [3, 5, 7, 9, 11],
    'min_child_weight': [1, 3, 5, 7, 9],
    'subsample': [0.7, 0.8, 9.0, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0, 1.2, 1.4],
    'gamma': [0, 0.1, 0.2, 0.3],
    'scale_pos_weight': [1, 3, 5, 7]
}
mod_gscv_xgb = GridSearchCV(estimator = mod_xgb_model, param_grid = xgb_param_grid, cv = 5, n_jobs = -1)
mod_gscv_xgb.fit(X_train_scaled, y_train)

/home/sanven0212/PycharmProjects/MultiOmicsFYP/.venv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
198000 fits failed out of a total of 360000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
54000 fits failed with the following error:
Traceback (most recent call last):
  File "/home/sanven0212/PycharmProjects/MultiOmicsFYP/.venv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/sanven0212/PycharmProjects/MultiOmicsFYP/.venv/lib/python3.10/site-packages/xgboost/core.py", line 726, in inner_f
    return func(**kwargs)
  File "/home/sanven0212/PycharmProjects/MultiOmicsFYP/.venv/lib/p

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.6, 0.8, 1.0, 1.2, 1.4],
                         'gamma': [0, 0.1, 0.2, 0.3],
                         'learning_rate': [0.001, 0.01, 0.1],
                         'max_depth': [3, 5, 7, 9, 11],
                         'min_child_weight': [1, 3, 5, 7, 9],
                         'n_estimators': [25, 50, 100],
                         'scale_pos_weight': [1, 3, 5, 7],
                         'subsample': [0.7, 0.8, 9.0, 1.0]})

In [27]:
print(f'Best parameters for XGBoost are: {mod_gscv_xgb.best_params_}')

Best parameters for XGBoost are: {'colsample_bytree': 0.6, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 5, 'n_estimators': 100, 'scale_pos_weight': 1, 'subsample': 0.7}


In [28]:
new_xgb = XGBClassifier(colsample_bytree = 0.6, gamma = 0, learning_rate = 0.1, max_depth = 5, min_child_weight = 5, n_estimators = 100, scale_pos_weight = 1, subsample = 0.7)
new_xgb.fit(X_train_scaled, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.6, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=5, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [29]:
print(classification_report(y_val, new_xgb.predict(X_val_scaled)))

              precision    recall  f1-score   support

           0       0.62      0.53      0.57        15
           1       0.56      0.64      0.60        14

    accuracy                           0.59        29
   macro avg       0.59      0.59      0.59        29
weighted avg       0.59      0.59      0.59        29



In [30]:
print(classification_report(y_test, new_xgb.predict(X_test_scaled)))

              precision    recall  f1-score   support

           0       0.72      0.56      0.63        32
           1       0.58      0.73      0.64        26

    accuracy                           0.64        58
   macro avg       0.65      0.65      0.64        58
weighted avg       0.66      0.64      0.64        58



In [31]:
print(classification_report(y_train, new_xgb.predict(X_train_scaled)))

              precision    recall  f1-score   support

           0       0.94      0.96      0.95        98
           1       0.96      0.94      0.95       105

    accuracy                           0.95       203
   macro avg       0.95      0.95      0.95       203
weighted avg       0.95      0.95      0.95       203



In [32]:
param_grid = {
    'n_estimators': [25, 50, 100, 250, 500, 1000],
    'learning_rate': [0.001, 0.01, 0.1],
    'max_depth': [3, 5, 7, 9, 11],
    'min_child_weight': [1, 3, 5, 7, 9],
    'subsample': [0.6, 0.8, 1.0, 1.2, 1.4],
    'colsample_bytree': [0.6, 0.8, 1.0, 1.2, 1.4]
}
xgbc = XGBClassifier()
gscv = GridSearchCV(estimator = xgbc, param_grid = param_grid, cv = 5, n_jobs = -1)
gscv.fit(X_train_scaled, y_train)

/home/sanven0212/PycharmProjects/MultiOmicsFYP/.venv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
36000 fits failed out of a total of 56250.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
6750 fits failed with the following error:
Traceback (most recent call last):
  File "/home/sanven0212/PycharmProjects/MultiOmicsFYP/.venv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/sanven0212/PycharmProjects/MultiOmicsFYP/.venv/lib/python3.10/site-packages/xgboost/core.py", line 726, in inner_f
    return func(**kwargs)
  File "/home/sanven0212/PycharmProjects/MultiOmicsFYP/.venv/lib/pyth

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.6, 0.8, 1.0, 1.2, 1.4],
                         'learning_rate': [0.001, 0.01, 0.1],
                         'max_depth': [3, 5, 7, 9, 11],
                         'min_child_weight': [1, 3, 5, 7, 9],
                         'n_estimators': [25, 50, 100, 250, 500, 1000],
                         'subsample': [0.6, 0.8, 1.0, 1.2, 1.4]})

In [33]:
print(f'Best params for XGBoost {gscv.best_params_}')

Best params for XGBoost {'colsample_bytree': 0.6, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 5, 'n_estimators': 25, 'subsample': 0.8}


In [34]:
new_xgb_model = XGBClassifier(colsample_bytree = 0.6, learning_rate = 0.1, max_depth = 3, min_child_weight = 5, n_estimators = 100, subsample = 0.8)
new_xgb_model.fit(X_train_scaled, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.6, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=5, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [35]:
print(classification_report(y_val, new_xgb_model.predict(X_val_scaled)))

              precision    recall  f1-score   support

           0       0.64      0.60      0.62        15
           1       0.60      0.64      0.62        14

    accuracy                           0.62        29
   macro avg       0.62      0.62      0.62        29
weighted avg       0.62      0.62      0.62        29



In [36]:
print(classification_report(y_test, new_xgb_model.predict(X_test_scaled)))

              precision    recall  f1-score   support

           0       0.78      0.66      0.71        32
           1       0.65      0.77      0.70        26

    accuracy                           0.71        58
   macro avg       0.71      0.71      0.71        58
weighted avg       0.72      0.71      0.71        58



In [37]:
print(classification_report(y_train, new_xgb_model.predict(X_train_scaled)))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98        98
           1       0.99      0.97      0.98       105

    accuracy                           0.98       203
   macro avg       0.98      0.98      0.98       203
weighted avg       0.98      0.98      0.98       203



In [38]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc_param_grid = {'max_depth': [2, 3, 5, 10, 20], 'min_samples_leaf': [5, 10, 20, 50, 100], 'criterion': ['gini', 'entropy']}
gscv_dtc = GridSearchCV(estimator = dtc, param_grid = dtc_param_grid, cv = 5, n_jobs = -1)
gscv_dtc.fit(X_train_scaled, y_train)


/home/sanven0212/PycharmProjects/MultiOmicsFYP/.venv/lib/python3.10/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 5, 10, 20],
                         'min_samples_leaf': [5, 10, 20, 50, 100]})

In [39]:
print(f'Best params for decision tree: {gscv_dtc.best_params_}')

Best params for decision tree: {'criterion': 'entropy', 'max_depth': 20, 'min_samples_leaf': 5}


In [40]:
dtc_2 = DecisionTreeClassifier(criterion = 'entropy', max_depth = 10, min_samples_leaf = 5)
dtc_2.fit(X_train_scaled, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=10, min_samples_leaf=5)

In [41]:
print(classification_report(y_val, dtc_2.predict(X_val_scaled)))

              precision    recall  f1-score   support

           0       0.62      0.53      0.57        15
           1       0.56      0.64      0.60        14

    accuracy                           0.59        29
   macro avg       0.59      0.59      0.59        29
weighted avg       0.59      0.59      0.59        29



In [42]:
print(classification_report(y_test, dtc_2.predict(X_test_scaled)))

              precision    recall  f1-score   support

           0       0.71      0.38      0.49        32
           1       0.51      0.81      0.63        26

    accuracy                           0.57        58
   macro avg       0.61      0.59      0.56        58
weighted avg       0.62      0.57      0.55        58



In [43]:
print(classification_report(y_train, dtc_2.predict(X_train_scaled)))

              precision    recall  f1-score   support

           0       0.90      0.72      0.80        98
           1       0.78      0.92      0.85       105

    accuracy                           0.83       203
   macro avg       0.84      0.82      0.82       203
weighted avg       0.84      0.83      0.83       203



In [44]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc_param_grid = {'n_estimators': [100, 200, 300, 500, 750, 1000], 'criterion': ['gini', 'entropy'], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'max_features': ['auto', 'sqrt', 'log2']}
gscv_rfc = GridSearchCV(estimator = rfc, param_grid = rfc_param_grid, cv = 5, n_jobs = -1)
gscv_rfc.fit(X_train_scaled, y_train)

/home/sanven0212/PycharmProjects/MultiOmicsFYP/.venv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
2160 fits failed out of a total of 6480.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1017 fits failed with the following error:
Traceback (most recent call last):
  File "/home/sanven0212/PycharmProjects/MultiOmicsFYP/.venv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/sanven0212/PycharmProjects/MultiOmicsFYP/.venv/lib/python3.10/site-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/home/sanven0212/PycharmProjects/MultiOmicsFYP/.venv/li

GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 10, 20, 30],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300, 500, 750, 1000]})

In [45]:
print(f'Best params for Random Forest: {gscv_rfc.best_params_}')

Best params for Random Forest: {'criterion': 'gini', 'max_depth': 30, 'max_features': 'log2', 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 100}


In [46]:
new_rfc = RandomForestClassifier(criterion = 'gini', max_depth = None, max_features = 'sqrt', min_samples_leaf = 2, min_samples_split = 2, n_estimators = 100)
new_rfc.fit(X_train_scaled, y_train)

RandomForestClassifier(min_samples_leaf=2)

In [47]:
print(classification_report(y_val, new_rfc.predict(X_val_scaled)))

              precision    recall  f1-score   support

           0       0.75      0.60      0.67        15
           1       0.65      0.79      0.71        14

    accuracy                           0.69        29
   macro avg       0.70      0.69      0.69        29
weighted avg       0.70      0.69      0.69        29



In [48]:
print(classification_report(y_test, new_rfc.predict(X_test_scaled)))

              precision    recall  f1-score   support

           0       0.71      0.62      0.67        32
           1       0.60      0.69      0.64        26

    accuracy                           0.66        58
   macro avg       0.66      0.66      0.65        58
weighted avg       0.66      0.66      0.66        58



In [49]:
print(classification_report(y_train, new_rfc.predict(X_train_scaled)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        98
           1       1.00      1.00      1.00       105

    accuracy                           1.00       203
   macro avg       1.00      1.00      1.00       203
weighted avg       1.00      1.00      1.00       203

